# Middleware: Human In The Loop
<img src="./assets/LC_HITL.png" width="300">



## Setup

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=<not set>
LANGSMITH_API_KEY=****4abe
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ad-6


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [ ]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime


@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If the database is offline, ask user to try again later without further comment.
"""

## Middleware

In [6]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="ollama:gpt-oss:20b",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver(),
    context_schema=RuntimeContext,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"execute_sql": {"allowed_decisions": ["approve", "reject"]}},
        ),
    ],
)

In [7]:
from langgraph.types import Command

question = "What are the names of all the employees?"

config = {"configurable": {"thread_id": "1"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
    context=RuntimeContext(db=db)
)

if "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m Interrupt:{description}\033[0m"
    )

    result = agent.invoke(
        Command(
            resume={
                "decisions": [{"type": "reject", "message": "the database is offline."}]
            }
        ),
        config=config,  # Same thread ID to resume the paused conversation
        context=RuntimeContext(db=db),
    )
    print(f"\033[1;3;31m{80 * '-'}\033[0m")

print(result["messages"][-1].content)

--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': 'SELECT name FROM employees LIMIT 5;'}
--------------------------------------------------------------------------------
I’m sorry, but the database is currently offline. Please try again later.


In [8]:
config = {"configurable": {"thread_id": "2"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
    context=RuntimeContext(db=db)
)

while "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m Interrupt:{description}\033[0m"
    )

    result = agent.invoke(
        Command(
            resume={"decisions": [{"type": "approve"}]}
        ),
        config=config,  # Same thread ID to resume the paused conversation
        context=RuntimeContext(db=db),
    )

for msg in result["messages"]:
    msg.pretty_print()

--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': 'SELECT name FROM employees'}
--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': "SELECT name FROM sqlite_master WHERE type='table'"}
--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': 'SELECT FirstName, LastName FROM Employee'}
================================ Human Message =================================

What are the names of all the employees?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (7070313a-bee5-4b01-88be-a4640680767a)
 Call ID: 7070313a-bee5-4b01-88be-a4640680767a
  Args:
    query: SELECT name FROM employees
===============================